In [1]:
import os
import sys
import yaml
import pickle
import glob
import copy

import pandas as pd

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import ProjectCard
from network_wrangler import Scenario
from network_wrangler import WranglerLogger

from lasso import Parameters

from network_wrangler.utils import create_unique_shape_id

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
logger.setLevel(logging.INFO)

In [4]:
# root_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'travel-model-two-networks')
root_dir = "D:/metcouncil_network_rebuild"
input_dir = os.path.join(root_dir, 'data', 'processed')
output_dir = os.path.join(root_dir, 'data', 'processed', 'version_01')
card_dir = os.path.join(root_dir, 'project_cards')
# lasso_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'mtc-Lasso')
lasso_dir = 'Z:/Data/Users/Sijia/Met_Council/github/client_met_council_wrangler_utilities'

In [5]:
parameters = Parameters(lasso_base_dir = lasso_dir)

2022-03-21 09:23:45, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/Met_Council/github/client_met_council_wrangler_utilities
2022-03-21 09:23:45, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/Met_Council/github/client_met_council_wrangler_utilities


In [6]:
version_11_pickle_file_name = os.path.join(
    input_dir, 
    'version_11',
    'working_scenario_01.pickle')
v_11_scenario = pickle.load(open(version_11_pickle_file_name, 'rb'))

In [7]:
version_12_pickle_file_name = os.path.join(
    input_dir, 
    'version_12', 
    'working_scenario_01.pickle')
v_12_scenario = pickle.load(open(version_12_pickle_file_name, 'rb'))

In [8]:
def create_ID_crosswalk(
    link_old,
    link_new,
    path
):
    """
    create the ID crosswalk between versions of network,
    in case the IDs are changed between version for the same geometries
    """
    
    link_old_df = link_old.copy()
    link_new_df = link_new.copy()

    
    link_old_df["shape_id"] = link_old_df["geometry"].apply(
                lambda x: create_unique_shape_id(x)
            )
    link_new_df["shape_id"] = link_new_df["geometry"].apply(
                lambda x: create_unique_shape_id(x)
            )
    link_join_df = pd.merge(
        link_old_df[["model_link_id", "A", "B","shape_id"]].rename(
            columns = {"model_link_id" : "model_link_id_old",
                       "A" : "A_old",
                       "B" : "B_old"}
        ),
        link_new_df[["model_link_id", "A", "B","shape_id"]].rename(
            columns = {"model_link_id" : "model_link_id_new",
                       "A" : "A_new",
                       "B" : "B_new"}
        ),
        how = "left",
        on = ["shape_id"]
    )
    
    link_join_df = link_join_df[
        (link_join_df.model_link_id_old != link_join_df.model_link_id_new) |
        (link_join_df.A_old != link_join_df.A_new) |
        (link_join_df.B_old != link_join_df.B_new)
    ]
    
    link_join_df[["model_link_id_old", "model_link_id_new"]].to_csv(
        os.path.join(path, "link_id_crosswalk.csv"),index=False)
    
    node_join_df = pd.concat(
        [link_join_df[["A_old", "A_new"]].rename(
            columns = {"A_old" : "model_node_id_old",
                       "A_new" : "model_node_id_new"}
        ),
         link_join_df[["B_old", "B_new"]].rename(
            columns = {"B_old" : "model_node_id_old",
                       "B_new" : "model_node_id_new"}
        )
        ],
        sort = False,
        ignore_index = True)
    
    node_join_df = node_join_df[
        node_join_df.model_node_id_old != node_join_df.model_node_id_new]
    node_join_df.drop_duplicates(inplace = True)
    
    node_join_df.to_csv(
        os.path.join(path, "node_id_crosswalk.csv"),index=False)
    
    return node_join_df,link_join_df

In [9]:
node_join_df, link_join_df = create_ID_crosswalk(
    link_old = v_11_scenario.road_net.links_df,
    link_new = v_12_scenario.road_net.links_df,
    path = output_dir
)

In [10]:
node_join_df[node_join_df.model_node_id_old == 1027709]

,model_node_id_old,model_node_id_new
95,1027709,1027710


In [6]:
crosswalk_df = pd.read_csv(os.path.join(root_dir, 'data', 'interim', 'model_network_id_crosswalk.csv'))

In [31]:
crosswalk_df[crosswalk_df.model_link_id_old == 398283]

,shstReferenceId,A_old,B_old,model_link_id_old,model_link_id_new,A_new,B_new


In [9]:
crosswalk_df[crosswalk_df['model_link_id_new'].isnull()]

,shstReferenceId,A_old,B_old,model_link_id_old,model_link_id_new,A_new,B_new
276,0c7a898b3f98eebc4bfcaf0dddb96105,3947,3949,1531,NaN,NaN,NaN
1628,2222fd97e3a2c17f16f128c9a6103b8a,5871,107024,4892,NaN,NaN,NaN
2652,1671603e1de63bc63c3f62c6770e01ce,6992,91463,7089,NaN,NaN,NaN
3853,fac0a5a4c1000b4b00170439e9aa1fee,8538,8542,10125,NaN,NaN,NaN
3854,df57138bdb1d24226e8ce0e7cdb1ec14,8542,8538,10132,NaN,NaN,NaN
...,...,...,...,...,...,...,...
466184,357c600eb98a225465097dad9a08e8cb,129776,80350,411992,NaN,NaN,NaN
466323,7caa24dca4e01ac6561f5c22d42a4b52,137895,137916,413700,NaN,NaN,NaN
466324,194aaddaa723ea672687eec2188916f7,137895,137916,413700,NaN,NaN,NaN
466325,48268c9fabde466247763a58eb4880a6,137895,76262,413701,NaN,NaN,NaN


In [12]:
crosswalk_df = crosswalk_df[crosswalk_df['model_link_id_new'].notnull()].copy()

In [13]:
crosswalk_df['model_link_id_new'] = crosswalk_df['model_link_id_new'].astype(int)

In [15]:
link_crosswalk = crosswalk_df.groupby(['model_link_id_old'])['model_link_id_new'].apply(list).reset_index()

link_crosswalk = dict(zip(link_crosswalk['model_link_id_old'], link_crosswalk['model_link_id_new']))

In [16]:
link_crosswalk

{1: [409420],
 3: [208243, 400359],
 4: [343056, 282309],
 6: [90253],
 7: [771172],
 8: [321093],
 9: [169687],
 12: [318570],
 13: [115644],
 14: [410498],
 18: [943181],
 19: [101107, 6733],
 20: [431453],
 22: [236240],
 23: [795383, 806949],
 24: [264002],
 26: [60108, 371141],
 27: [161715],
 28: [91509],
 29: [363994, 411025, 225533, 68168],
 30: [13626, 22626],
 31: [294239],
 32: [425469, 144409, 533327],
 33: [109665, 53465, 112902],
 34: [520562, 435266, 519095, 107600, 434610],
 35: [59698],
 36: [56906, 313618],
 37: [458138],
 38: [247868, 327341],
 39: [543978],
 40: [362872],
 41: [912396],
 42: [413793],
 43: [196420, 531270, 101137, 303830, 225306, 255161],
 44: [13561, 81857],
 45: [314642, 541995, 451583, 405177],
 46: [536301],
 47: [175556],
 48: [277118, 115190],
 49: [349043],
 50: [115809],
 51: [69529],
 52: [772671, 905580, 924557, 845581, 862004, 945340],
 53: [170357, 194705],
 55: [270359],
 56: [630577],
 57: [477531, 430658, 438941],
 58: [482574],
 59: 

# update project cards with new IDs

In [17]:
import glob

In [18]:
suffix = ["*.yaml", "*.yml", "*.wrangler", "*.wr"]
card_list = []
for s in suffix:
    card_list.extend(glob.glob(os.path.join(card_dir, s)))

In [19]:
len(card_list)

33

In [14]:
node_crosswalk = dict(
)

link_crosswalk = dict(
    zip(link_join_df.model_link_id_old, link_join_df.model_link_id_new)
)

In [25]:
def map_new_id(x, crosswalk):
    return crosswalk.get(x)

def transit_id_change(d):
    for p in d.get('properties'):
        if p.get('property') == 'routing':
            _existing = []
            _set = []
            for n in p.get('existing'):
                if abs(n) in list(node_crosswalk.keys()):
                    if n > 0:
                        node = map_new_id(n, node_crosswalk)
                    else:
                        node = -(map_new_id(abs(n), node_crosswalk))
                else:
                    node = n
                _existing.append(node)
            for n in p.get('set'):
                if abs(n) in list(node_crosswalk.keys()):
                    if n > 0:
                        node = map_new_id(n, node_crosswalk)
                    else:
                        node = -(map_new_id(abs(n), node_crosswalk))
                else:
                    node = n
                _set.append(node)
                
            p['existing'] = _existing
            p['set'] = _set

    return d

def roadway_id_change(d):
    if d.get('category') in [
        'Roadway Property Change', 'Parallel Managed lanes'
    ]:

        for link in d.get('facility').get('link'):
            if "model_link_id" in list(link.keys()):
                
                _model_link_id_list = []
                for l in link.get('model_link_id'):
                    if l in list(link_crosswalk.keys()):
                        #_model_link_id_list.append(
                        #    map_new_id(l, link_crosswalk)
                        #)
                        _model_link_id_list = _model_link_id_list + map_new_id(l, link_crosswalk)
                    else:
                        None

                link['model_link_id'] = _model_link_id_list
        
        return d
    
    if d.get('category') in ['Add New Roadway']:
        for link in d.get('links'):
            if link['A'] in list(node_crosswalk.keys()):
                link['A'] = map_new_id(link['A'], node_crosswalk)
            if link['B'] in list(node_crosswalk.keys()):
                link['B'] = map_new_id(link['B'], node_crosswalk)
        
        return d
    
    if d.get('category') in ['Roadway Deletion']:
        _model_link_id_list = []
        for l in d.get('links').get('model_link_id'):
            if l in list(link_crosswalk.keys()):
                link = map_new_id(l, link_crosswalk)

            else:
                None
            #_model_link_id_list.append(link)
            _model_link_id_list = _model_link_id_list + link

        d['links']['model_link_id'] = _model_link_id_list
        
        return d
        

def update_id(card):
    
    for change in card.changes:
        if change.get('category') == 'Transit Service Property Change':
            change = transit_id_change(change)

        else:
            change = roadway_id_change(change)

    return card

def update_project_cards(
    path,
):
    suffix = ["*.yaml", "*.yml"]
    card_list = []
    for s in suffix:
        card_list.extend(glob.glob(os.path.join(path, s)))
        
    for file in card_list:
        card = ProjectCard.read(
            file,
            validate = False
        )
        
        card = update_id(card)
        card.__dict__.pop('file', None)
        card.__dict__.pop('valid', None)
        
        card.write(
            filename = file
        )

In [29]:
update_project_cards(path = card_dir)

2022-03-21 10:11:03, INFO: Wrote project card to: D:/metcouncil_network_rebuild\project_cards\AsgnGrp1.yml
2022-03-21 10:11:03, INFO: Wrote project card to: D:/metcouncil_network_rebuild\project_cards\AsgnGrp1.yml
2022-03-21 10:11:04, INFO: Wrote project card to: D:/metcouncil_network_rebuild\project_cards\AsgnGrp15.yml
2022-03-21 10:11:04, INFO: Wrote project card to: D:/metcouncil_network_rebuild\project_cards\AsgnGrp15.yml
2022-03-21 10:11:05, INFO: Wrote project card to: D:/metcouncil_network_rebuild\project_cards\AsgnGrp2.yml
2022-03-21 10:11:05, INFO: Wrote project card to: D:/metcouncil_network_rebuild\project_cards\AsgnGrp2.yml
2022-03-21 10:11:07, INFO: Wrote project card to: D:/metcouncil_network_rebuild\project_cards\AsgnGrp4.yml
2022-03-21 10:11:07, INFO: Wrote project card to: D:/metcouncil_network_rebuild\project_cards\AsgnGrp4.yml
2022-03-21 10:11:07, INFO: Wrote project card to: D:/metcouncil_network_rebuild\project_cards\AsgnGrp50.yml
2022-03-21 10:11:07, INFO: Wrote p